# Segmenting and Clustering Neighborhoods in Toronto
### CAPSTONE PROJECT WEEK 3

For this assignment, we will be required to explore and cluster the neighborhoods in Toronto.

Start by creating a new Notebook for this assignment.


Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

* ##### Use beautifulsoup to scrape the postal code information from the wikipedia link given above

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, "lxml")
table = soup.find("table")
table_rows = table.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

ConnectionError: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /wiki/List_of_postal_codes_of_Canada:_M (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fac68dce510>: Failed to establish a new connection: [Errno -2] Name or service not known'))

* ##### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood


In [ ]:
df = pd.DataFrame(res, columns=["Postal Code", "Borough", "Neighborhood"])
df.dtypes

* ##### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.


In [ ]:
df = df[(df.Borough != 'Not assigned')] #& (df.Neighborhood !='None')]
df.head()

* ##### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be  the same as the borough.

The neighborhoods with same postalcode have already been merged in the wikipedia page. Hence we only replace '/' with ',' to get the desired dataframe


In [ ]:
df['Neighborhood'] = df['Neighborhood'].str.replace('/', ',')
df.head()

In [ ]:
df.shape

In [ ]:
import geocoder
from geopy.geocoders import Nominatim
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
import folium

In [ ]:
df_latlng = pd.read_csv('Geospatial_Coordinates.csv')
df_latlng.shape

In [ ]:
df_loc = pd.merge(df, df_latlng, on='Postal Code')
df_loc.head()

In [ ]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_loc['Latitude'], df_loc['Longitude'], df_loc['Borough'], df_loc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Add Foursqare credentials to find information about the locations

In [ ]:
CLIENT_ID = 'WV1R2FKGLBLY0U4HFLDNUQZTPO2JWJQG3WQVRCM12HZTWZMI'
CLIENT_SECRET = 'YLYT3CGV0BDUKDUQRVAW5Z3BKLGIOGBCFCTUJHWAL0IQGZF2'
VERSION = '20200417' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url